## Robot setup

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("..")
sys.path.append("../../")

from armControl import *
from forwardKinematics import robot_config
from utils.motor_setup import maxonGearSmall
import armControl

import signal

socket_ip = '192.168.1.26'
socket_port = 1124

#trajPlanner = trajectoryGenerator()

motors = maxonGearSmall()
motors.tcp_init(socket_ip, socket_port)

#axis_motor_indexes = np.array([-1, -1, -1, 0, 3, 2, 1]) #indexed from 0 for motors and axis
#velocity = np.ones(8)*3.14/20


Opening socket at ip: 192.168.1.26 using port: 1124


## Command motors w/ indexing for joints

def commandJoints(setpoint_arm, trajectory = True):
    myArm = armControl.remoteRobotArm()
    myArm.jointAngleSetpoint = setpoint_arm
    myArm.updateMotorArmMixing()
    setpoint_motor = np.zeros(8)
    setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
    if trajectory:
        motors.run_trajectory(setpoint_motor, velocity)
    else:
        motors.command_motors_radians(setpoint_motor)


## Arm Motors

In [16]:
print("Arming motors now...")
motors.arm_motors()

Arming motors now...
initializing motors to [   -1     0     1     0  1193 -9102     0     0]
*** C side has an error or needs to be armed ***

Arming motors


## Manual motor zeroing

In [20]:
from importlib import reload
reload(armControl)

myArm = armControl.remoteRobotArm()
setpoint_arm = np.array([-82.5, 0,0,0,0,0,0])
myArm.commandJoints(motors, setpoint_arm)

In [19]:
for i in range(8):
    motors.zero_motors_radians(i, motors.get_motors_position_radians()[i])

## Test arm joint mixing

### this is old, now in "commandJoints" above

import armControl
from importlib import reload
reload(armControl)

setpoint_arm = np.array([0,0,0,0,0,0,-0])
myArm = armControl.remoteRobotArm()
myArm.jointAngleSetpoint = setpoint_arm
myArm.updateMotorArmMixing()
print(myArm.motorAngleSetpoint)

setpoint_motor = np.zeros(8)
setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
print(setpoint_motor)
motors.run_trajectory(setpoint_motor, velocity)


In [23]:
setpoint_arm = np.array([0,0,0,0.,0,0.5,0])
myArm.commandJoints(motors, setpoint_arm)

## Teleop testing

## Sine wave / profile test

In [30]:
dt = 0.005
runtime = 10
num_elements = int(runtime/dt)
setpoints = np.zeros((num_elements, 7))
frequency = 0.1
amplitude = np.pi/6
for i in range(num_elements):
    setpoints[i,:] = (np.ones(7) * np.sin(i*dt*2*pi*frequency)*amplitude)


start_time = time.time()

currents_mA = []
for i in range(num_elements):
    current_time = time.time()
    setpoint = setpoints[i]
    myArm.commandJoints(motors, setpoint, trajectory=False)
    time.sleep(dt)

time.sleep(1.5)

motors.run_trajectory(np.zeros(8), velocity)


0

## Cleanup

In [14]:
setpoints = np.zeros(8)
velocity = np.ones(8)*3.14/2
motors.run_trajectory(setpoints, velocity)

motors.tcp_close()

OSError: [Errno 9] Bad file descriptor

In [45]:
setpoints = np.zeros(8)
velocity = np.ones(8)*3.14/2
motors.run_trajectory(setpoints, velocity)


0